In [1]:
import pandas as pd
import numpy as np


In [2]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

/Users/vastav/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
calls_df = pd.read_csv('../data/raw/support_calls.csv')

In [5]:
def label_churn_risk(text):
    if isinstance(text, str):
        text_lower = text.lower()
        if any(keyword in text_lower for keyword in ["cancel", "closing", "leave", "unhappy"]):
            return 1
    return 0

calls_df['churn_label'] = calls_df['call_transcript'].apply(label_churn_risk)

print("Distribution of churn_label:\n", calls_df['churn_label'].value_counts())

Distribution of churn_label:
 churn_label
0    16804
1     3196
Name: count, dtype: int64


In [6]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

texts = calls_df['call_transcript'].tolist()
labels = calls_df['churn_label'].tolist()

Downloading: 100%|██████████| 232k/232k [00:00<00:00, 622kB/s] 
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 85.1kB/s]
Downloading: 100%|██████████| 483/483 [00:00<00:00, 1.17MB/s]


In [7]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

import torch
class ChurnDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = ChurnDataset(train_encodings, train_labels)
val_dataset = ChurnDataset(val_encodings, val_labels)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Downloading: 100%|██████████| 268M/268M [00:11<00:00, 22.8MB/s] 
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly init

In [9]:
trainer.train()

/Users/vastav/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2000
  Number of trainable parameters = 66955010
  2%|▎         | 50/2000 [01:07<41:12,  1.27s/it] 

{'loss': 0.2669, 'learning_rate': 4.899497487437186e-05, 'epoch': 0.03}


  5%|▌         | 100/2000 [02:32<1:02:05,  1.96s/it]

{'loss': 0.0035, 'learning_rate': 4.7738693467336685e-05, 'epoch': 0.05}


  8%|▊         | 150/2000 [03:45<44:09,  1.43s/it]  

...

100%|██████████| 2000/2000 [47:46<00:00,  1.23s/it]Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json


{'loss': 0.0039, 'learning_rate': 0.0, 'epoch': 1.0}


Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 8
                                                   
100%|██████████| 2000/2000 [51:16<00:00,  1.23s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 2000/2000 [51:16<00:00,  1.54s/it]

{'eval_loss': 5.013004192733206e-05, 'eval_runtime': 208.6831, 'eval_samples_per_second': 19.168, 'eval_steps_per_second': 2.396, 'epoch': 1.0}
{'train_runtime': 3076.3433, 'train_samples_per_second': 5.201, 'train_steps_per_second': 0.65, 'train_loss': 0.022710809982614592, 'epoch': 1.0}


TrainOutput(global_step=2000, training_loss=0.022710809982614592, metrics={'train_runtime': 3076.3433, 'train_samples_per_second': 5.201, 'train_steps_per_second': 0.65, 'train_loss': 0.022710809982614592, 'epoch': 1.0})

In [10]:
val_preds = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_preds.predictions, axis=1)

***** Running Prediction *****
  Num examples = 4000
  Batch size = 8
100%|██████████| 500/500 [04:02<00:00,  2.07it/s]


In [11]:
print("\nClassification Report:")
print(classification_report(val_labels, val_pred_labels))

print("\nNLP-based Churn Prediction Complete.")


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3377
           1       1.00      1.00      1.00       623

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


NLP-based Churn Prediction Complete.
